In [1]:
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

import pandas as pd
from functions import dirs, readFile, canonicalNames
from wordEmbedders import WESCScore
import pickle
import chevron
from re import sub
from latexTable import LatexTable

In [3]:
#datasets = dirs('../data')
names = canonicalNames('../data')
dimensions = pickle.load(open('../data/Dimensions-All.pickle', mode="rb"))

In [11]:
alldims = [item for sublist in dimensions.values() for item in sublist]

In [60]:
for dataset in datasets:
    dimsFile = f"../data/{dataset}/Dimensions.csv"
    dims = pd.read_csv(dimsFile)
    desc = dims.describe()
    def val(dim, v):
        return f"{desc[dim][v]:.2f}"
    rows = []
    for aspect in [x for x in dimensions.keys() if not x == 'postag']:
        n = len(dimensions[aspect])
        items = list(map(lambda dim: ["", dim, val(dim, 'min'), val(dim, 'mean'), val(dim, 'max'), val(dim, 'std')], dimensions[aspect]))
        items[0][0] = "\parbox[t]{2mm}{\multirow{"+str(n)+"}{*}{\\rotatebox[origin=c]{90}{"+aspect+"}}}"
        rows += items
    table = LatexTable()
    table.headers = ['Variable', 'Min', 'Mean', 'Max', 'STD']
    table.columnAlignments = ['l','l','r','r','r','r']
    table.emptyFirstHeader = True
    table.boldIndexColumn = False
    table.rows = rows
    result = table.render()
    rows = result.split("\n")
    out = []
    for i, r in enumerate(rows):
        if i + 1 < len(rows):
            nxt = rows[i+1]
        else:
            nxt = None
        if nxt is None or nxt[0] == ' ':
            r = sub(r"\\hline", r"\\cline{2-6}", r)
        out.append(r)
    #result = sub(r"\\hline", r"\\cline{2-6}", result)
    #print("\n".join(out))
    with open(f'./Appendix-Variables-{dataset}.tex',mode="w") as f:
        f.write("\n".join(out))